# How to get Minimal Voting Weights and Power Indices of Country Parliaments

This code is built around data from the political yearbook (https://politicaldatayearbook.com) but can be used for any other weighted voting games.
If you use political yearbook data just use the download as csv option. The code currently assumes that you have a folder called data in the same directory from which you execute this notebook and that the csv_files are generally named "country_name.csv"
Everything in here can be run as is, however if you want to save your results, you need to pip install xslxwriter.   

In [4]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import glob
import time 
import numpy as np
import ast 
from datetime import datetime
from itertools import combinations 
from mwc_functions import *
from mwc_class import getMVWs
from optimization_functions import *
from power_indice_functions import * 
from government_functions import * 



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## To get the results of a single country over multiple years execute the following cell: 

In [ ]:
start_time=time.time()
xy_data = getMVWs('country_name.csv', name='country_name', save_results=True,verify_mwcs=True,encoding='UTF-8',delimiter=',',find_all_weights=True,find_errors=False,results_folder='results')
country_prelims = xy_data.preliminaries()
country_MIW = xy_data.minimal_voting_weights_pipeline()
country_power_indices = xy_data.power_indices_pipeline()
end_time=time.time()
duration=end_time-start_time

print(f'{xy_data.name} took {duration} seconds.')

the class getMVWs needs as inputs:
- the file name, a str
- the name of the country, game etc, a str

the class getMVWs accepts the following other keywords: 
- save_results, boolean whether you want results saved to excel files
- results_folder, a str creates if necessary the specified folder and saves the results there
- encoding, a str to specify the encoding of the csv file (political yearbook uses untypical 'utf-16')
- delimiter, a str to specify the symbol used to seperate columns in the csv file (political yearbook uses untypical '\t')

the class getMVWs also uses the following boolean keywords to indicate preffered funcitonality, setting these to True considerably prolongs the algorithm (unrecommended for n>20)
- find all_weights, boolean whether the program should look for non unique Minimal Sum Representations (main runtime cost)
- verify_mwcs, boolean whether after Minimal Sum Representation has been found the program should verify that these represent the same game as the original weights
- find_errors, boolen generally only for error_tracking 

The class getMVWs then allows for the use of its 3 main methods: 
- self.preliminaries()
- self.minimal_voting_weights_pipeline()
- self.power_indices_pipeline()

which should always be called in that order. 
Comments: 
- Preliminaries typically runs very quick but can produce big Excel files (~20 MB for the parliaments of Spain i.e)
- minimal_voting_weights_pipeline uses the most runtime and uses the results from "preliminaries" to calculate Minimal Voting Weights 
    - Minimal Winning and Maximal Losing Coalitions are used to generate constraints for the linear integer program 
    - Constraints are passed to scipy.optimize.milp
    - if verify_mwcs then optimized results are used to generate a new dict of all possible coalitions and verify it being identical to the one created in preliminaries 
    - if find_all_weights: 
        - optimized results are extracted and all combinations with same weight-sum but weight changes up to +1 (-1) are generated 
        - each combination is a possible other optimal result as such, weights are added to the constraints with strict equality 
        - scipy.optimize.milp is called for each new set of constraints and checks whether the problem is still feasible 
        - collects all resulting optimal set of weights 
- power_indices_pipeline uses the minimal weights from minimal_voting_weights_pipeline to calculate: 
    - Banzhaf (non-normalized) Index values 
    - Shapely Shubik Index Values
    - Minimal Voting Weights Index Values (following Freixas&Kaniovski 2014 )
    - the first two use the same algorithms as the 'powerindices' package and I refer to the documentation on https://github.com/frankhuettner/powerindices and credit for the algorithms goes to Hüttner, Frank.
    - for every tested parliament it runs very quickly

# Replication of Data used in the Thesis:
- all data was downloaded from https://politicaldatayearbook.com/
- for the example of Austria: https://politicaldatayearbook.com/ChartDataCsv.aspx?chartGroup=ELECTION_RESULTS&countryId=99&electionTypeId=1&round=null
- csv file was stored in a folder 'data' and renamed accordingly, for this example: 'austria.csv'

Changes made to the csv of israel: 
- 1999: one instance of "israel our home" changed to "NatU-NRP - National Union/NRP (National Union/NRP, NatU-NRP)" following https://ejpr.onlinelibrary.wiley.com/doi/10.1111/j.1475-6765.2000.tb01150.x

Changes made to the csv of poland: 
- 1991: removed this election from the dataset. More than 30 parties were elected into parliament which is not solvable with this code. However one could use the results from: https://ejpr.onlinelibrary.wiley.com/doi/10.1111/j.1475-6765.1992.tb00339.x

the following function allows to extract the country name from the csv file name (used for the coming loop)

In [5]:
# Loop for all countries:
def country_name_from_file(file_path):
    #gets the name of the country from the currently 'inspected' file
    #helper function for next cell
    file = os.path.basename(file_path)
    country_name, _ = os.path.splitext(file)
    return country_name.capitalize()


folder = 'data/'
cases_to_be_looked_at = {}


The following loop goes over all csv files in the 'data' folder and executes the above explained pipeline for every one of them. Results are stored in a folder 'results' as Excel files.
For all countries from the political yearbook with the aforementioned changes this takes about two hours (mainly driven by Italy (30 minutes) and Spain (70 minutes), also Isreal and Switzerland took about 10 minutes). 


In [7]:
start_time = time.time()

for csv_file in glob.glob(os.path.join(folder, '*.csv')):
    country_name = country_name_from_file(csv_file)
    country_start_time = time.time()
    country_data = getMVWs(f'{country_name}.csv', name=country_name, save_results=True,verify_mwcs=True)
    prelims = country_data.preliminaries()
    country_MIW = country_data.minimal_voting_weights_pipeline()
    country_power_indices = country_data.power_indices_pipeline()
    country_end_time = time.time()
    country_duration = country_end_time-country_start_time
    print(f'{country_name} took {country_duration} seconds')
    questionable_years=[]
    for year,dict in country_data.errors.items(): 
        if dict: 
            questionable_years.append(year)
    if questionable_years: 
        cases_to_be_looked_at[country_name]=questionable_years        
end_time = time.time()
total_time = end_time - start_time
print(f"Total time: {total_time:.2f} seconds")

An error occurred: UTF-16 stream does not start with BOM
Krohn took 1.233902931213379 seconds
Latvia took 0.23589301109313965 seconds
Lithuania took 13.677065134048462 seconds
Luxembourg took 0.15014052391052246 seconds
Malta took 0.10218286514282227 seconds
Netherlands took 21.3109872341156 seconds
Newzealand took 0.29064345359802246 seconds
Norway took 0.3837265968322754 seconds
Poland_without91 took 0.219163179397583 seconds
Portugal took 0.25417518615722656 seconds
Romania took 0.12732625007629395 seconds
Slovakia took 0.30017733573913574 seconds
Slovenia took 0.501544713973999 seconds
Spain took 4282.541248321533 seconds
Sweden took 1.1027333736419678 seconds
Switzerland took 762.6199374198914 seconds
United-kingdom took 2.001326084136963 seconds
Usa took 0.13063883781433105 seconds
Total time: 5087.20 seconds


You should check that the cases_to_be_looked_at dict is empty if you used verify_mwcs=True. This dict would list all country-years where any errors occured. 

# Create final df:

In [ ]:

def country_df(file_name): 
    df=pd.read_excel(f'mvws/{file_name}')
    ###get country name part###
    first_split=file_name.split('-')
    country_name=first_split[1].split('.')[0]
    ### get dates from excel###
    dates=df['Key_1'].unique()
    dates_dt_1= pd.to_datetime(dates,errors='coerce',format='-%b-%y').sort_values()
    dates_dt_2= pd.to_datetime(dates,errors='coerce',format='%Y-%m').sort_values() #sometimes date format changes
    new_df=pd.DataFrame()
    if not dates_dt_2.empty:       ## if date format changed we need this convoluted bit here:
        dates_dt_2_1=dates_dt_2.strftime('%Y-%m')
        dates_dt_2_formatted=dates_dt_2.strftime('-%b-%y')

        sort_dates_str_1=dates_dt_1.strftime('-%b-%y')

        for i,date in enumerate(dates_dt_2_1):
            relevant_rows=df[df['Key_1']==date]
            parties=relevant_rows['Key_2'].tolist()
            mvws=relevant_rows['Value_1'].tolist()
            new_df[i]=pd.Series([parties,mvws])
            new_df.rename(columns={i:dates_dt_2_formatted[i]},inplace=True)

    ### create df as wanted
    for date in sort_dates_str_1:
        relevant_rows=df[df['Key_1']==date]
        parties=relevant_rows['Key_2'].tolist()
        mvws=relevant_rows['Value_1'].tolist()
        new_df[date]=pd.Series([parties,mvws])
    new_df = new_df.loc[:, new_df.columns.notna()]        
    new_df=new_df.add_prefix(f'{country_name}')
    return new_df


In [ ]:
def update_prelims(file_name,all_countries_df): 
    file= pd.ExcelFile(f'prelims/{file_name}')
    ### Part for getting the original Seats
    df= file.parse('Transformed Data')
    dates=df['YearMonth'].unique()
    country_name=file_name.split('-')[1].split('.')[0]
    formatted_dates=[f"{country_name}{date}" for date in dates]
    
    for date in formatted_dates: 
        try: 
            if date in all_countries_df.columns:
                relevant_rows= df[df['YearMonth']==date.replace(f'{country_name}','')] 
                parties_from_file= relevant_rows['Party'].tolist()
                parties_from_df=all_countries_df.loc['parties',date]
                if parties_from_df==parties_from_file: 
                    if 'Seats' not in all_countries_df.index:
                        all_countries_df.loc['Seats']=[np.nan]*len(all_countries_df.columns)
                    seats= relevant_rows['# of Seats'].tolist()
                    all_countries_df.at['Seats',date]= seats
                else: all_countries_df.at['Seats', date] = 'error'
            else: raise ValueError
        except ValueError: 
            country_part=date[:-7]
            date_part=date[-7:]
            date_part_dt=datetime.strptime(date_part, '%Y-%m')
            date_part=date_part_dt.strftime('-%b-%y')
            date_correctformat=f'{country_part}{date_part}'
        
            if date_correctformat in all_countries_df.columns:
                relevant_rows= df[df['YearMonth']==date.replace(f'{country_name}','')] 
                parties_from_file= relevant_rows['Party'].tolist()
                parties_from_df=all_countries_df.loc['parties',date_correctformat]
                if parties_from_df==parties_from_file: 
                    if 'Seats' not in all_countries_df.index:
                        all_countries_df.loc['Seats']=[np.nan]*len(all_countries_df.columns)
                    seats= relevant_rows['# of Seats'].tolist()
                    all_countries_df.at['Seats',date_correctformat]= seats
                else: 
                    all_countries_df.at['Seats', date] = 'error'
            else: print(f'something is wrong with {date}')
    ### part for getting n_in_year:
    n_df=file.parse('n per Year')
    for index,row1 in n_df.iterrows(): 
        date=row1['Key']
        formatted_date=f'{country_name}{date}'
        try:
            if formatted_date in all_countries_df.columns: 
                if 'n per Year' not in all_countries_df.index: 
                    all_countries_df.loc['n per Year']=[np.nan]*len(all_countries_df.columns)
                all_countries_df.at['n per Year',formatted_date]= row1['Value_1']
            else: raise ValueError
        except ValueError: 
            date_part_dt=datetime.strptime(date, '%Y-%m')
            date_part=date_part_dt.strftime('-%b-%y')
            date_correctformat=f'{country_name}{date_part}'
            if date_correctformat in all_countries_df.columns: 
                if 'n per Year' not in all_countries_df.index: 
                    all_countries_df.loc['n per Year']=[np.nan]*len(all_countries_df.columns)
                all_countries_df.at['n per Year',date_correctformat]= row1['Value_1']
    ### part for getting Total Seats: 
    Q_df=file.parse('Total Seats per Year')
    for index,row1 in Q_df.iterrows(): 
        date=row1['Key']
        formatted_date=f'{country_name}{date}'
        try:
            if formatted_date in all_countries_df.columns: 
                if 'Q' not in all_countries_df.index: 
                    all_countries_df.loc['Q']=[np.nan]*len(all_countries_df.columns)
                all_countries_df.at['Q',formatted_date]= row1['Value_1']
            else: raise ValueError
        except ValueError: 
            date_part_dt=datetime.strptime(date, '%Y-%m')
            date_part=date_part_dt.strftime('-%b-%y')
            date_correctformat=f'{country_name}{date_part}'
            if date_correctformat in all_countries_df.columns: 
                if 'Q' not in all_countries_df.index: 
                    all_countries_df.loc['Q']=[np.nan]*len(all_countries_df.columns)
                all_countries_df.at['Q',date_correctformat]= row1['Value_1']
    return all_countries_df

In [ ]:
def update_powerindices(file_name,all_countries_df): 
    file=pd.ExcelFile(f'powerindices/{file_name}')
    for sheet_name in file.sheet_names: 
        df=pd.read_excel(file,sheet_name=sheet_name)
        country_name=file_name.split('-')[1].split('.')[0]
        date=f'{country_name}{sheet_name}'
        try:
            if date in all_countries_df: 
                parties_from_sheet=df['Party'].tolist()
                parties_from_df=all_countries_df.loc['parties',date]
                if parties_from_sheet==parties_from_df: 
                    for col in df.columns[1:]: 
                        if col not in all_countries_df.index: 
                            all_countries_df.loc[col] = [np.nan] * len(all_countries_df.columns)    
                        all_countries_df.at[col, date] = df[col].tolist()
                else: 
                    all_countries_df.at[col, date] = 'error' 
                    print(parties_from_sheet)
            else: raise ValueError
        except:
            date_part_dt=datetime.strptime(sheet_name,'%Y-%m')
            date_part=date_part_dt.strftime('-%b-%y')
            date_correctformat = f'{country_name}{date_part}'
            
            if date_correctformat in all_countries_df.columns:
                parties_from_sheet=df['Party'].tolist()
                parties_from_df=all_countries_df.loc['parties',date_correctformat]
                if parties_from_sheet==parties_from_df: 
                    for col in df.columns[1:]: 
                        if col not in all_countries_df.index: 
                            all_countries_df.loc[col] = [np.nan] * len(all_countries_df.columns)    
                        all_countries_df.at[col, date_correctformat] = df[col].tolist() 
            else:  print(f'something is wrong with {date} into {date_correctformat}')
                    
                
            
    return all_countries_df

In [ ]:
all_mvw_files=[i for i in os.listdir('mvws') if i.endswith('.xlsx')]
all_originalseats_files=[i for i in os.listdir('prelims') if i.endswith('.xlsx')]
all_powerfiles=[i for i in os.listdir('powerindices') if i.endswith('.xlsx')]

In [ ]:
country_dfs = []
for mvw_file in all_mvw_files: 
    df=country_df(mvw_file)
    country_dfs.append(df)
all_countries_df= pd.concat(country_dfs,axis=1)
indexnames= ['parties','mvws']
all_countries_df.index = indexnames
for prelimfile in all_originalseats_files: 
    all_countries_df=update_prelims(prelimfile,all_countries_df)
for powerfile in all_powerfiles: 
    all_countries_df=update_powerindices(powerfile,all_countries_df)

## Add Government Data to elections: 

In [ ]:
final_df=pd.read_excel('results/complete_dataframe.xlsx',index_col=0)

In [ ]:

for csv_file in glob.glob(os.path.join('government/','*.csv')): 
    file= os.path.basename(csv_file)
    countryname,_=os.path.splitext(file)
    #create country government dataframe
    df=process_gov_csv(csv_file)
    #grab part of final_df that is relevant for the country
    country_df=final_df[[col for col in final_df.columns if col.startswith(countryname)]] #this is case sensitive... 
    elections=country_df.columns.to_numpy() # list of elections in the country
    #get all election_df's (currently not really necessary but might be if stability or changes in government becomre relevant)
    election_period_dict=match_ministries_and_elections(countryname,elections,df)
    #subset election_df's for only the first ministers in an election period (initial government)
    government_dict=starting_gov_dict(election_period_dict)
    ##loop over all election periods: ##
    for date,dataframe in government_dict.items():
        if not dataframe.empty:
            party_str=country_df.at['parties',f'{countryname}{date}']
            parties=ast.literal_eval(party_str) # parses string to list  
            #create dicts
            ministry_dict,unweighted_dict,weighted_dict=get_ministry_dicts(dataframe,parties) 
            #translate dicts into lists / arrays corrosponding to the party list     
            ministy_list=[]
            unweighted_array=np.zeros(len(parties))  
            weighted_array=np.zeros(len(parties))  
            for i,(party,value) in enumerate(ministry_dict.items()): 
                ministy_list.append(value)
            for i,(party,value) in enumerate(unweighted_dict.items()):
                unweighted_array[i]=value
            for i,(party,value) in enumerate(weighted_dict.items()):
                weighted_array[i]=value
            
            # write list/arrays to final_df:
            if 'Ministers' not in final_df.index: 
                final_df.loc['Ministers']=[np.nan] * len(final_df.columns)   
            if 'unweighted' not in final_df.index: 
                final_df.loc['unweighted']=[np.nan] * len(final_df.columns)   
            if 'weighted' not in final_df.index: 
                final_df.loc['weighted']=[np.nan] * len(final_df.columns) 
            final_df.at['Ministers',f'{countryname}{date}']=ministy_list
            final_df.at['unweighted',f'{countryname}{date}']=unweighted_array
            final_df.at['weighted',f'{countryname}{date}']=weighted_array 



    



In [ ]:
with pd.ExcelWriter('combined_dataframe.xlsx', engine='xlsxwriter') as writer:
    final_df.to_excel(writer)